## 1. Transfer Learning , 모델 불러오기

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout, Flatten
from tensorflow.keras.applications import VGG16, ResNet50

In [ ]:
base_model = tf.keras.applications.VGG16(weights='imagenet',
                                        include_top=False,
                                        input_shape=(112,112,3))

In [ ]:
base_model.summary()

In [ ]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(64,activation='relu'),
    Dropout(0.3)
    Dense(4,activation='softmax')
])

In [ ]:
model.summary()

## 2. ImageDataGenerator 사용해서 증강해보기

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img = image.load_img("/content/고양이.jpg",target_size=(150,150))

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
datagen = ImageDataGenerator(
    rotation_range = 90,
    width_shift_range=0.4,
    zoom_range=0.4,
    # vertical_flip=True,
)

In [ ]:
datagen

In [ ]:
image_batch =np.expand_dims(img, axis=0)

In [ ]:
image_batch.shape

In [ ]:
np.array(img).shape

In [ ]:
idx = 0
fig = plt.figure(figsize=(10,10))
axs = []
for batch in datagen.flow(image_batch,batch_size=1):
    axs.append(fig.add_subplot(5,2,idx+1))
    axs[idx].imshow(image.array_to_img(batch[0]))
    idx += 1
    if idx%10 == 0:
        break
fig.tight_layout()
plt.show()


## 3. 개/고양이 데이터셋 불러오기

In [ ]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

In [ ]:
import os, zipfile
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout ,Input
from tensorflow.keras.applications import ResNet50, MobileNet
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
device = tf.test.gpu_device_name() #/device:GPU:0
print(device)

In [ ]:
with zipfile.ZipFile('/content/cats_and_dogs_filtered.zip','r') as z:
    z.extractall()

In [ ]:
print("학습용 고양이 갯수",len(os.listdir("/content/cats_and_dogs_filtered/train/cats")))
print("학습용 강아지 갯수",len(os.listdir("/content/cats_and_dogs_filtered/train/dogs")))
print("평가용 고양이 갯수",len(os.listdir("/content/cats_and_dogs_filtered/validation/cats")))
print("평가용 강아지 갯수",len(os.listdir("/content/cats_and_dogs_filtered/validation/dogs")))

In [ ]:
pwd #print working directory

In [ ]:
cd cats_and_dogs_filtered/

In [ ]:
root_dir = "/content/cats_and_dogs_filtered"

In [ ]:
train_dir = os.path.join(root_dir, 'train_set')
test_dir = os.path.join(root_dir, 'test_set')
print(train_dir)
print(test_dir)
os.mkdir(train_dir)
os.mkdir(test_dir)

In [ ]:
test_cat = os.path.join(test_dir, 'cat')
test_dog = os.path.join(test_dir, 'dog')
print(test_cat)
print(test_dog)

os.mkdir(test_cat)
os.mkdir(test_dog)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                   rotation_range=40,
                   width_shift_range=0.2,
                   shear_range=0.2,
                   zoom_range=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory("/content/cats_and_dogs_filtered/train_2",
                                                    target_size=(150,150),
                                                    batch_size=32)
test_generator = test_datagen.flow_from_directory("/content/cats_and_dogs_filtered/validation",
                                                    target_size=(150,150),
                                                    batch_size=32)

In [ ]:
base_model = tf.keras.applications.VGG16(weights='imagenet',
                                         include_top=False,
                                         input_shape=(150,150,3))

In [ ]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dropout(0.25),
    Dense(2, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
             loss='categorical_crossentropy',
              metrics=['accuracy'])

earlystopping = EarlyStopping(monitor='val_loss', patience=5)


hist = model.fit_generator(train_generator,
                   epochs=5,
                   validation_data=test_generator,
                   callbacks=[earlystopping])